In [1]:
def to_categorical(y):
    from keras import backend as k
    return k.one_hot(y, 512)

In [2]:
from keras.layers import Input, Dense, Concatenate,Lambda,Reshape,Dropout, BatchNormalization
from keras.models import Model
import numpy as np

vector = Input(shape=(512,), name='Vectors-Input',dtype='float32')
vector_shiter0 = Dense(512)(vector)
norm_vector_shiter0=BatchNormalization()(vector_shiter0)
vector_shiter = Dense(512)(norm_vector_shiter0)
norm_vector_shiter=BatchNormalization()(vector_shiter)
position = Input(shape=(1,), name='Positions-Input',dtype='int32')
categorical_position=Lambda(to_categorical, name='Positional_encoding')(position)
reshaped_categorical_position=Reshape((512,))(categorical_position)
concatenated_encoder_input=Concatenate()([norm_vector_shiter,reshaped_categorical_position])
encoder_input_divider1 = Dense(1024, name='Encoder-Output-Divider-1',activation='selu')(concatenated_encoder_input)
norm_encoder_input1=BatchNormalization()(encoder_input_divider1)

encoder_input_divider2 = Dense(512, name='Encoder-Output-Divider-2',activation='selu')(norm_encoder_input1)
norm_encoder_input2=BatchNormalization()(encoder_input_divider2)

output_vector = Dense(512, name='Encoder-Garbedge',activation='selu')(norm_encoder_input2)
encoder_input_divider3 = Dense(256, name='Encoder-Output-Divider-3',activation='selu')(norm_encoder_input2)
norm_encoder_input3=BatchNormalization()(encoder_input_divider3)

output_vector2 = Dense(100, name='Encoder-Output',activation='selu')(norm_encoder_input3)

model = Model(inputs=[vector,position],outputs=[output_vector,output_vector2])
model.compile(
    optimizer='adam',
    loss='mean_absolute_error',
)

Using TensorFlow backend.
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [3]:
from bpemb import BPEmb
from collections import Iterable
import numpy as np

bpemb_ru = BPEmb(lang="ru", dim=100, vs=3000)

def textreader(st):
    texts = []
    with open("D:/decoder/data/data3000/np_rev2.txt", "r", encoding='utf-8') as fr:
        i=0
        for line in fr:
            i+=1
            if i<st:      
                continue
            texts.append(line.replace('\n',''))
            if i==st+499:
                break
    return texts

def word_vectors_creator(data):
    l=len(data)
    kol= np.zeros(shape=(l))
    pieces=[]
    for i in range(l):
        s = bpemb_ru.encode_ids(data[i])
        kol[i]+=len(s)
        pieces.append(s)
    kol = kol.astype(np.int)
    out = np.zeros(shape=(np.sum(kol),100))
    #out2 = np.zeros(shape=(np.sum(kol),100))
    buf = 0
    for g in range(l):        
        for k in range(0,kol[g]):
            out[buf]+=bpemb_ru.emb.vectors[pieces[g][k]]
            '''
            for m in range(kol[g]):
                if m!=k:
                    out2[buf]+=(bpemb_ru.emb.vectors[pieces[g][m]]/kol[g])
            buf+=1
            '''
    return out, kol       #, out2

def vectorsextender(data,kol):
    out = np.zeros(shape=(np.sum(kol),512))
    out2 = np.zeros(shape=(np.sum(kol)))
    buf=0
    for g in range(data.shape[0]):
        for k in range(kol[g]):
            out[buf]+=data[g]
            #out2[buf]+=np.sum(data[g])/512
            buf+=1
    return out#, out2

def positioncreator(kol):
    out = np.zeros(shape=(np.sum(kol)))
    buf=0
    for g in range(len(kol)):
        out[buf:buf+kol[g]]+=np.arange(1,kol[g]+1)
    return out
    

In [3]:
from keras.models import load_model
model = load_model ('D:/112/decoder/models/mse_sen_to_word0_100.h5')

Using TensorFlow backend.
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [4]:
from sklearn.utils import shuffle

start=1
n=501
e=502
for i in range(101,200):
    vectors_train = np.load('D:/decoder/data/data3000/news_vectors'+str(999-i)+'.npy')
    #print(start)
    texts = textreader(start)
    start+=500
    #print(len(texts))

    for j in range(5):
        tt, kol  = word_vectors_creator(texts[0+j*100:(1+j)*100])
        vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
        pos = positioncreator(kol)
        tt, vtt,pos = shuffle(tt, vtt,pos, random_state=0)

        if n%20==0:
            model.fit(
                x=[vtt,
                   pos],
                y = [vtt,
                    tt],
                epochs=e, initial_epoch=n,
                validation_split=0.05,
                verbose=1,
                batch_size=4
            )
            if n%500==0:
                model.save('D:/decoder/models/mse_sen_to_word'+str(d)+'_'+str(i)+'.h5')
                print('\n')
        else:
            model.fit(
                x=[vtt,
                   pos],
                y = [vtt,
                    tt],
                epochs=e, initial_epoch=n,
                validation_split=0.05,
                verbose=0,
                batch_size=4
            )
        n+=1
        e+=1

Train on 4578 samples, validate on 241 samples
Epoch 521/521
4578/4578 [==============================] - 17s 4ms/step - loss: 7.8072 - Encoder-Garbedge_loss: 0.0132 - Encoder-Output_loss: 7.7940 - val_loss: 1.9301 - val_Encoder-Garbedge_loss: 0.0138 - val_Encoder-Output_loss: 1.9163
Train on 5409 samples, validate on 285 samples
Epoch 541/541
5409/5409 [==============================] - 21s 4ms/step - loss: 7.8029 - Encoder-Garbedge_loss: 0.0131 - Encoder-Output_loss: 7.7898 - val_loss: 1.8076 - val_Encoder-Garbedge_loss: 0.0128 - val_Encoder-Output_loss: 1.7947
Train on 3873 samples, validate on 204 samples
Epoch 561/561
3873/3873 [==============================] - 14s 4ms/step - loss: 6.8133 - Encoder-Garbedge_loss: 0.0130 - Encoder-Output_loss: 6.8003 - val_loss: 1.7637 - val_Encoder-Garbedge_loss: 0.0120 - val_Encoder-Output_loss: 1.7517
Train on 6058 samples, validate on 319 samples
Epoch 581/581
6058/6058 [==============================] - 22s 4ms/step - loss: 8.3074 - Encoder-G

In [4]:
from sklearn.utils import shuffle


n=0
e=1
for d in range(2):
    start=1
    print('Epoch: '+str(d)+'\n'+"**********************************\n")
    for i in range(200):
        vectors_train = np.load('D:/decoder/data/data3000/news_vectors'+str(999-i)+'.npy')
        #print(start)
        texts = textreader(start)
        start+=500
        #print(len(texts))

        for j in range(5):
            tt, kol  = word_vectors_creator(texts[0+j*100:(1+j)*100])
            vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
            pos = positioncreator(kol)
            tt, vtt,pos = shuffle(tt, vtt,pos, random_state=0)

            if n%20==0:
                model.fit(
                    x=[vtt,
                       pos],
                    y = [vtt,
                        tt],
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=1,
                    batch_size=8
                )
                if n%500==0:
                    model.save('D:/decoder/models/mae_sen_to_word'+str(d)+'_'+str(i)+'.h5')
                    print('\n')
            else:
                model.fit(
                    x=[vtt,
                       pos],
                    y = [vtt,
                        tt],
                    epochs=e, initial_epoch=n,
                    validation_split=0.05,
                    verbose=0,
                    batch_size=8
                )
            n+=1
            e+=1
    model.save('D:/decoder/models/mae_sen_to_word'+str(d)+'.h5')
    print('\n\n\n')

Epoch: 0
**********************************


Train on 4308 samples, validate on 227 samples
Epoch 1/1
4308/4308 [==============================] - 27s 6ms/step - loss: 0.2586 - Encoder-Garbedge_loss: 0.1298 - Encoder-Output_loss: 0.1288 - val_loss: 0.0991 - val_Encoder-Garbedge_loss: 0.0637 - val_Encoder-Output_loss: 0.0354


Train on 4538 samples, validate on 239 samples
Epoch 21/21
4538/4538 [==============================] - 10s 2ms/step - loss: 0.0753 - Encoder-Garbedge_loss: 0.0322 - Encoder-Output_loss: 0.0431 - val_loss: 0.0407 - val_Encoder-Garbedge_loss: 0.0286 - val_Encoder-Output_loss: 0.0121
Train on 4973 samples, validate on 262 samples
Epoch 41/41
4973/4973 [==============================] - 11s 2ms/step - loss: 0.0622 - Encoder-Garbedge_loss: 0.0264 - Encoder-Output_loss: 0.0358 - val_loss: 0.0341 - val_Encoder-Garbedge_loss: 0.0218 - val_Encoder-Output_loss: 0.0124
Train on 4933 samples, validate on 260 samples
Epoch 61/61
4933/4933 [==============================] - 1

Train on 4759 samples, validate on 251 samples
Epoch 561/561
4759/4759 [==============================] - 9s 2ms/step - loss: 0.0489 - Encoder-Garbedge_loss: 0.0211 - Encoder-Output_loss: 0.0278 - val_loss: 0.0139 - val_Encoder-Garbedge_loss: 0.0137 - val_Encoder-Output_loss: 2.0658e-04
Train on 3977 samples, validate on 210 samples
Epoch 581/581
3977/3977 [==============================] - 7s 2ms/step - loss: 0.0494 - Encoder-Garbedge_loss: 0.0212 - Encoder-Output_loss: 0.0282 - val_loss: 0.0141 - val_Encoder-Garbedge_loss: 0.0139 - val_Encoder-Output_loss: 2.0803e-04
Train on 4747 samples, validate on 250 samples
Epoch 601/601
4747/4747 [==============================] - 8s 2ms/step - loss: 0.0508 - Encoder-Garbedge_loss: 0.0214 - Encoder-Output_loss: 0.0293 - val_loss: 0.0143 - val_Encoder-Garbedge_loss: 0.0141 - val_Encoder-Output_loss: 2.1480e-04
Train on 3945 samples, validate on 208 samples
Epoch 621/621
3945/3945 [==============================] - 7s 2ms/step - loss: 0.0506 - E

KeyboardInterrupt: 

In [6]:
model.save('D:/decoder/models/sen_to_word'+str(i)+'.h5')